In [1]:
import os
import sys
sys.path.insert(0, '../src/')

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz, save_npz

import torch

import utils
from net import start_experiments
from baselines import (Forge_Adjacency, Forge_Modularity, Forge_SymmetricLaplacian, Forge_Transition,
                       train_and_log_multiple_conf_models, Transition)

In [2]:
#logroot = '../logs'
logroot = '../rel_error_table'

In [3]:
dataset = 'rt_gop'
#dataset = 'CORA-ML'
_A_obs = load_npz(f'../data/datasets/{dataset}.npz')

In [4]:
val_share = 0.1
test_share = 0.05
seed = 481516234

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False)

train_graph = sp.csr_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1])))
assert (train_graph.toarray() == train_graph.toarray().T).all()
val_edges = (test_ones, test_zeros)

## Forge Framework

In [5]:
fa = Forge_Adjacency(A=train_graph, rank=1600)
graphs_fa = fa(sample_size=5,
               logdir=os.path.join(logroot, dataset, 'baseline_FA'),
               val_edges=val_edges)

In [6]:
ft = Forge_Transition(A=train_graph, rank=1600)
graphs_ft = ft(sample_size=5,
               logdir=os.path.join(logroot, dataset, 'baseline_FT'),
               val_edges=val_edges)

In [7]:
fm = Forge_Modularity(A=train_graph, rank=1600)
graphs_fm = fm(sample_size=5,
               logdir=os.path.join(logroot, dataset, 'baseline_FM'),
               val_edges=val_edges)

In [8]:
fs = Forge_SymmetricLaplacian(A=train_graph, rank=2500)
graphs_fs = fs(sample_size=5,
               logdir=os.path.join(logroot, dataset, 'baseline_FS'),
               val_edges=val_edges)

## Transition with Cross Entropy

In [9]:
models = start_experiments(num_experiments=5,
                           experiment_root=os.path.join(logroot, dataset, 'baseline_transition_ce_loss'),
                           train_graph=train_graph,
                           H=950,
                           optimizer=torch.optim.Adam,
                           optimizer_args={'lr': 0.1,
                                           'weight_decay': 1e-7},
                           invoke_every=5,
                           steps = 100,
                           Model=Transition,
                           val_edges=(val_ones, val_zeros))


Experiment_0
Step:   5/100, Loss: 8.20919, Edge-Overlap: 0.000
Step:  10/100, Loss: 8.00024, Edge-Overlap: 0.001
Step:  15/100, Loss: 7.81789, Edge-Overlap: 0.001
Step:  20/100, Loss: 7.60926, Edge-Overlap: 0.003
Step:  25/100, Loss: 7.36899, Edge-Overlap: 0.003
Step:  30/100, Loss: 7.08921, Edge-Overlap: 0.003
Step:  35/100, Loss: 6.76065, Edge-Overlap: 0.006
Step:  40/100, Loss: 6.38176, Edge-Overlap: 0.012
Step:  45/100, Loss: 5.95989, Edge-Overlap: 0.016
Step:  50/100, Loss: 5.50552, Edge-Overlap: 0.021
Step:  55/100, Loss: 5.03098, Edge-Overlap: 0.038
Step:  60/100, Loss: 4.54356, Edge-Overlap: 0.073
Step:  65/100, Loss: 4.04784, Edge-Overlap: 0.104
Step:  70/100, Loss: 3.56227, Edge-Overlap: 0.160
Step:  75/100, Loss: 3.10808, Edge-Overlap: 0.249
Step:  80/100, Loss: 2.70426, Edge-Overlap: 0.332
Step:  85/100, Loss: 2.37319, Edge-Overlap: 0.463
Step:  90/100, Loss: 2.13999, Edge-Overlap: 0.553
Step:  95/100, Loss: 2.00640, Edge-Overlap: 0.619
Step: 100/100, Loss: 1.93822, Edge-O

## Configuration Model

In [10]:
train_and_log_multiple_conf_models(logdir=os.path.join(logroot, dataset, 'baseline_conf'),
                                   A=train_graph,
                                   EOs=[0.3, 0.5, 0.7],
                                   experiments_per_EO=20)